In [1]:
"""
Alunos:
    David Cardoso Yonekura
    Lucas da Silva Lima
    Rafael Barbosa de Carvalho

"""

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from neuron import Neuron

In [2]:
with open('data/dataHoldout.txt', 'rb') as file:
    auxFile = file.read()


dataHoldout = np.frombuffer(auxFile)
dataHoldout.resize((int(len(dataHoldout)/3)), 3)

X = dataHoldout[...,0:2] #[[x01,x02], ...]
y = dataHoldout[...,2] #[[yd0], [yd1], ...]
print(y.dtype)
# #adicionando x_0 no x(n)
X = np.insert(X, 2, 1, axis=1)
#instanciando o neuronio, inserindo apenas a taxa de aprendizado
# e os intervalos superior e inferior dos pesos
config= [[0.01,-100,100], [0.01,-1,1], [0.01,-0.5,0.5]]
fitWeight = []
epoch = []
print("Configuração: ", config)
n = Neuron(config[0], config[1], config[2])
n.run100epochs(X, y)

float64
Configuração:  [[0.01, -100, 100], [0.01, -1, 1], [0.01, -0.5, 0.5]]
[0.01, -1, 1] [0.01, -0.5, 0.5]
[-0.29322959 -0.09065359  1.        ]
[0.07988839 0.21101297 1.        ]
[-0.07825563 -0.08083512  1.        ]
[-0.46833414 -0.10620722  1.        ]
[0.46304142 0.09405471 1.        ]
[ 0.0114998  -0.05939114  1.        ]
[0.27238274 0.18359397 1.        ]
[0.08690974 0.00654493 1.        ]
[0.0824192  0.00106606 1.        ]
[-0.00940128 -0.10654841  1.        ]
[ 0.05786483 -0.36451414  1.        ]
[0.07028775 0.39056305 1.        ]
[ 0.20013586 -0.41657686  1.        ]
[-0.06282733 -0.32877261  1.        ]
[-0.10391518  0.34638192  1.        ]
[0.25527419 0.17171587 1.        ]
[ 0.28382866 -0.00689202  1.        ]
[0.34015635 0.06961302 1.        ]
[-0.19908835  0.12926358  1.        ]
[-0.03197237  0.16450966  1.        ]
[-0.01465206  0.12680101  1.        ]
[0.29560862 0.08406349 1.        ]
[0.2367801  0.37531654 1.        ]
[-0.09127282  0.28768163  1.        ]
[-0.20389

TypeError: can't multiply sequence by non-int of type 'numpy.float64'

In [ ]:
x_min, x_max = X[:, 0].min() - 0.1, X[:, 0].max() + 0.1
y_min, y_max = X[:, 1].min() - 0.1, X[:, 1].max() + 0.1
xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.001),
                     np.arange(y_min, y_max, 0.001))

f, axarr = plt.subplots(1, 1, sharex='col', sharey='row', figsize=(10, 8))
color = np.where(y == 0, 'r', 'b')

for idx, clf, tt in zip([[0,0]],
                        [n],
                        ['logistic regression']):

    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = np.array(Z)
    Z = Z.reshape(xx.shape)

    axarr.contourf(xx, yy, Z, alpha=0.2,cmap='Greys')
    axarr.scatter(X[:, 0], X[:, 1], c=color, s=20, edgecolor='k',cmap='Greys')
    axarr.set_title(tt)


plt.show()

# f.savefig('gaph2.png')
# plt.close(f)

#exportando a figura para por no markdown